In [1]:
import geopandas as gpd
from shapely.geometry import Point

# Load the dataset
from geopandas.datasets import get_path


In [2]:
import pandas as pd

In [36]:
countries = pd.read_csv("../data/country_info.csv")
countries

,name,code,capital
0,Albania,al,Tirana
1,Armenia,am,Yerevan
2,Australia,au,Canberra
3,Austria,at,Vienna
4,Azerbaijan,az,Baku
5,Belgium,be,Brussels
6,Croatia,hr,Zagreb
7,Cyprus,cy,Nicosia
8,Czechia,cz,Prague
9,Denmark,dk,København


In [21]:
votes_df = pd.read_csv("../data/vote_distances.csv")

gdp_per_capita = pd.read_csv("../data/gdp_per_capita.csv").dropna()
gdp_per_capita = gdp_per_capita[gdp_per_capita["Series Name"] == "GDP per capita (current US$)"] 
year_cols = ["2019 [YR2019]","2020 [YR2020]","2021 [YR2021]","2022 [YR2022]","2023 [YR2023]"]
for col in year_cols:
    gdp_per_capita[col] = pd.to_numeric(gdp_per_capita[col], errors='coerce').dropna()


gdp_per_capita

,Series Name,Series Code,Country Name,Country Code,2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
4700,GDP per capita (current US$),NY.GDP.PCAP.CD,Albania,ALB,5460.428237,5370.778623,6413.283286,6846.426143,8575.171134
4701,GDP per capita (current US$),NY.GDP.PCAP.CD,Andorra,AND,41257.804585,37361.090067,42425.699676,42414.059009,46812.448449
4702,GDP per capita (current US$),NY.GDP.PCAP.CD,Armenia,ARM,4597.228874,4268.680933,4685.179971,6571.974455,8053.010663
4703,GDP per capita (current US$),NY.GDP.PCAP.CD,Australia,AUS,54972.701789,51791.540180,60607.778861,64997.013654,64820.911789
4704,GDP per capita (current US$),NY.GDP.PCAP.CD,Austria,AUT,49885.994736,48716.409890,53648.719074,52176.664914,56033.573792
...,...,...,...,...,...,...,...,...,...
4795,GDP per capita (current US$),NY.GDP.PCAP.CD,Ukraine,UKR,3619.861572,3709.769287,4775.945801,4199.670898,5069.703125
4796,GDP per capita (current US$),NY.GDP.PCAP.CD,United Kingdom,GBR,42662.535374,40201.681242,46896.866243,45935.927647,49463.855462
4797,GDP per capita (current US$),NY.GDP.PCAP.CD,United States,USA,65604.681520,64411.373178,71318.307359,78035.175360,82769.412211
4798,GDP per capita (current US$),NY.GDP.PCAP.CD,Norway,NOR,76430.588947,68340.018103,93072.892512,108798.451166,87925.094419


In [25]:
gdp_per_capita = gdp_per_capita.rename(columns={"Country Name": "country"})
gdp_per_capita["average gdp per capita"] = gdp_per_capita[year_cols].mean(axis=1)



KeyError: "None of [Index(['2019 [YR2019]', '2020 [YR2020]', '2021 [YR2021]', '2022 [YR2022]',\n       '2023 [YR2023]'],\n      dtype='object')] are in the [columns]"

In [38]:
gdp_per_capita = gdp_per_capita[["country", "average gdp per capita"]]

gdp_per_capita.head()
gdp_per_capita.loc[gdp_per_capita["country"] == "Russian Federation", "country"] = "Russia"
filtered_gdp_per_capita = gdp_per_capita[gdp_per_capita["country"].isin(countries["name"])]

# Display the filtered DataFrame
print(filtered_gdp_per_capita)

                     country  average gdp per capita
4700                 Albania             6533.217485
4701                 Andorra            42054.220357
4702                 Armenia             5635.214979
4703               Australia            59437.989254
4704                 Austria            52092.272481
4705              Azerbaijan             5868.042505
4707                 Belarus             7338.810601
4708                 Belgium            49957.362538
4709  Bosnia and Herzegovina             7168.458183
4717                 Croatia            17698.752213
4718                  Cyprus            32515.344922
4719                 Czechia            27021.094649
4720                 Denmark            65332.587659
4727                 Estonia            26898.692668
4729                 Finland            50730.033153
4730                  France            41815.398376
4732                 Georgia             5827.852688
4733                 Germany            50259.

In [40]:
filtered_gdp_per_capita["country"] = filtered_gdp_per_capita["country"].replace(
    dict(zip(countries["name"], countries["code"])))

filtered_gdp_per_capita 

C:\Users\Valentin\AppData\Local\Temp\ipykernel_2896\2515068596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_gdp_per_capita["country"] = filtered_gdp_per_capita["country"].replace(


,country,average gdp per capita
4700,al,6533.217485
4701,ad,42054.220357
4702,am,5635.214979
4703,au,59437.989254
4704,at,52092.272481
4705,az,5868.042505
4707,by,7338.810601
4708,be,49957.362538
4709,ba,7168.458183
4717,hr,17698.752213


In [33]:
votes_df


,Unnamed: 0,year,round,from_country,to_country,total_points,tele_points,jury_points,from_capital,to_capital,distance
0,0,1957,final,at,nl,6,NaN,NaN,Vienna,Amsterdam,935.926496
1,1,1957,final,at,fr,0,NaN,NaN,Vienna,Paris,1036.040752
2,2,1957,final,at,dk,0,NaN,NaN,Vienna,København,871.969165
3,3,1957,final,at,lu,3,NaN,NaN,Vienna,Luxembourg,765.856559
4,4,1957,final,at,de,0,NaN,NaN,Vienna,Berlin,524.812724
...,...,...,...,...,...,...,...,...,...,...,...
51349,51349,2023,final,wld,al,6,6.0,NaN,NaN,Tirana,NaN
51350,51350,2023,final,wld,pt,0,0.0,NaN,NaN,Lisbon,NaN
51351,51351,2023,final,wld,rs,0,0.0,NaN,NaN,Belgrade,NaN
51352,51352,2023,final,wld,gb,0,0.0,NaN,NaN,London,NaN


In [47]:
votes_with_gdp = votes_df.merge(filtered_gdp_per_capita, how="left", left_on="to_country", right_on="country")
# Adjust display options
pd.set_option('display.max_rows', 100)   # Show up to 100 rows
pd.set_option('display.max_columns', 50)
votes_with_gdp[votes_with_gdp["from_country"] == "de"]

,Unnamed: 0,year,round,from_country,to_country,total_points,tele_points,jury_points,from_capital,to_capital,distance,country,average gdp per capita
40,40,1957,final,de,nl,1,NaN,NaN,Berlin,Amsterdam,576.965942,nl,58172.028219
41,41,1957,final,de,fr,6,NaN,NaN,Berlin,Paris,879.117320,fr,41815.398376
42,42,1957,final,de,dk,0,NaN,NaN,Berlin,København,355.611123,dk,65332.587659
43,43,1957,final,de,lu,0,NaN,NaN,Berlin,Luxembourg,603.339626,lu,123405.563253
44,44,1957,final,de,de,0,NaN,NaN,Berlin,Berlin,0.000000,de,50259.725386
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50751,50751,2023,final,de,al,8,8.0,0.0,Berlin,Tirana,1333.628210,al,6533.217485
50752,50752,2023,final,de,pt,0,0.0,0.0,Berlin,Lisbon,2306.345222,pt,24461.189472
50753,50753,2023,final,de,rs,3,0.0,3.0,Berlin,Belgrade,1000.174032,rs,9567.869045
50754,50754,2023,final,de,gb,0,0.0,0.0,Berlin,London,933.332847,gb,45032.173194


In [ ]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [28]:
vote_dists = merged_votes.merge(distance_stack, left_on=["from_capital", "to_capital"], right_on=["from", "to"], how="left")
vote_dists = vote_dists.drop(["from", "to"], axis=1)
vote_dists

,year,round,from_country,to_country,total_points,tele_points,jury_points,from_capital,to_capital,distance
0,1957,final,at,nl,6,NaN,NaN,Vienna,Amsterdam,935.926496
1,1957,final,at,fr,0,NaN,NaN,Vienna,Paris,1036.040752
2,1957,final,at,dk,0,NaN,NaN,Vienna,København,871.969165
3,1957,final,at,lu,3,NaN,NaN,Vienna,Luxembourg,765.856559
4,1957,final,at,de,0,NaN,NaN,Vienna,Berlin,524.812724
...,...,...,...,...,...,...,...,...,...,...
51349,2023,final,wld,al,6,6.0,NaN,NaN,Tirana,NaN
51350,2023,final,wld,pt,0,0.0,NaN,NaN,Lisbon,NaN
51351,2023,final,wld,rs,0,0.0,NaN,NaN,Belgrade,NaN
51352,2023,final,wld,gb,0,0.0,NaN,NaN,London,NaN


In [29]:
vote_dists.to_csv("data/vote_distances.csv")